# Examen 15 novembre 2017

L'examen portait sur les données du fichier `csv` provenant de https://www.data.gouv.fr/fr/datasets/fichier-des-equides/

## Extraction des données

La difficulté principale provenait de la taille du fichier (et du fait que l'encodage ne soit pas précisé, qu'il ne s'agisse pas d'UTF-8 et que votre enseignant ne s'en soit même pas aperçu :shame:).
<img src="http://d3lp4xedbqa8a5.cloudfront.net/s3/digital-cougar-assets/Cosmo/2016/04/19/76052/cersei.jpg" width="200" height="40">


Surtout il faut éviter de lire plusieurs fois le fichier mais ça vous le savez déjà, n'est-ce pas ?

Pour être moins encombré pendant vos tests vous pouvez travailler dans un premier temps sur un échantillon de données. `head -n 3000 fichier.csv > fichier_short.csv` dans la console et voilà.

La grande question c'est quelles seront mes structures de données ? Ici l'objet sera un peu overkill mais rien ne vous empêche d'en utiliser. Vous pouvez choisir de conserver toutes les propriétés (les colonnes) des équidés ou de vous restreindre à celles dont vous avez besoin pour répondre aux questions.
Ce sera l'option choisie ici.

In [1]:
!ln -s mon_fichier_equides.csv Equides_2016.csv

ln: impossible de créer le lien symbolique 'Equides_2016.csv': Le fichier existe


In [ ]:
from collections import defaultdict
from collections import Counter
import re

file_name = "Equides_2016.csv"
poneys = [] # liste de tuples (name, sex)
autres = defaultdict(int) # name:freq

with open(file_name, "r", encoding="iso-8859-1") as f:
    for line in f:
        cols = line.split(",")
        if len(cols) != 8:
            continue
        # uniquement les poneys
        if "PONEY" in cols[0]:
            #on conserve le nom et le sexe de chaque poney dans un tuple
            poneys.append((cols[5], cols[1])) 
        # uniquement les pas poneys nés en G-B
        elif "GRANDE-BRETAGNE" in cols[4]:
            autres[cols[0]] += 1


Utiliser des fonctions n'était pas obligatoire mais commme plusieurs questions demandent de renvoyer les n plus fréquents, l'utilisation d'une fonction est tout indiquée.  

On peut faire le comptage et le filtre sur les n plus fréquents avec un dictionnaire (ou defaultdict) ou un objet Counter.

In [ ]:
def get_most_freq(data, n):
    """
    Finds the n most frequent items in data
    Args:
        data: list of tuples (name, sex)
        n: n most common items needed
    
    Returns:
        A list of tuples (name, freq)
    """
    data_dict = defaultdict(int)
    for name, sex in data:
        data_dict[name] += 1
    most_freq = sorted(data_dict, key=data_dict.get, reverse=True)
    return [(name, data_dict[name]) for name in most_freq[:n]]

In [ ]:
def get_most_freq_c(data, n):
    """
    Finds the n most frequent items in data (Counter version)
    Args:
        data: list of tuples (name, sex)
        n: n most common items needed
    
    Returns:
        A list of tuples (name, freq)
    """
    cnt = Counter([name for name,sex in data])
    #for name, sex in data:
    #    cnt[name] += 1
    return cnt.most_common(n)


Avec %timeit (commande magique d'ipython pour invoquer le module timeit), on peut tester sans efforts le temps de traitement de chaque fonction

In [ ]:
%timeit res = get_most_freq(poneys, 30)

In [ ]:
%timeit res = get_most_freq_c(poneys, 30)

### 1. Les trente noms de poneys les plus fréquents :

In [ ]:
for poney in get_most_freq_c(poneys, 30):
    print("{} {}".format(poney[0], poney[1]))


### 2. Noms se terminant par 'a', 'ah' ou 'at'

In [ ]:
pat = re.compile("A[HT]?$")
poneys_a = [poney for poney in poneys if pat.search(poney[0])]
for poney in get_most_freq_c(poneys_a, 10):
    print("{} {}".format(poney[0], poney[1]))

### 3. Noms se terminant par une consonne suivie de "ine"

In [ ]:
pat = re.compile("[^AEIOUY]INE$")
poneys_ine = [poney for poney in poneys if pat.search(poney[0])]
for poney in get_most_freq_c(poneys_ine, 10):
    print("{} {}".format(poney[0], poney[1]))

### 4. Nom le plus fréquent pour un mâle

In [ ]:
poneys_m = [poney for poney in poneys if poney[1] == "M"]
for poney in get_most_freq_c(poneys_m, 1):
    print("{} {}".format(poney[0], poney[1]))


### 4bis. Nom le plus fréquent pour une femelle

In [ ]:
poneys_m = [poney for poney in poneys if poney[1] == "F"]
for poney in get_most_freq_c(poneys_m, 1):
    print("{} {}".format(poney[0], poney[1]))

## Les autres

### 5. Trouver les chevaux (pas les poneys) nés en Grande-Bretagne et les compter par race. Afficher le décompte par ordre décroissant

In [ ]:
# les chevaux nés en G-B par race en ordre décroissant
most_freq = sorted(autres, key=autres.get, reverse=True)
for autre in most_freq[:10]:
    print("{} : {}".format(autre, autres[autre]))